# 1. Installation

In [1]:
!pip install --upgrade pip
!pip install llmcompressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 12.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 15.3 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 15.5 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: nvidia-ml-py
    Found existing installation: nvidia-ml-py 13.590.48
    Uninstalling nvidia-ml-py-13.590.48:
      Successfully uninstalled nvidia-ml-py-13.590.480/9 [nvidia-ml-py]
  Attempting uninstall: dill0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/9 [pyarrow]
    Found existing installation: dill 0.4.1━━━━━━━━━━━━━━━━━━━ 2/9 [pyarrow]
    Uninstalling dill-0.4.1:0m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/9 [dill]
      Successfully uninstalled dill-0.4.1━━━━━━━━━━━━━━━━━━━━━ 3/9 [dill]
  Attempting 

In [4]:
from huggingface_hub import login

!pip install ipywidgets

login()     # Your huggingface access token.

# 2. Compress Model

## 2.1. Quantization

See [Quantization Schemes](https://docs.vllm.ai/projects/llm-compressor/en/0.8.1/guides/compression_schemes/) for choosing quantization method.

In [ ]:
# INT W8A8 quantization with SmoothQuant and GPTQ
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot


recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]

oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir="TinyLlama-1.1B-Chat-v1.0-INT8",
    max_seq_length=2048,
    num_calibration_samples=512,
)

In [1]:
# Inference quantized model via vLLM
from vllm import LLM, SamplingParams


model_path = "./TinyLlama-1.1B-Chat-v1.0-INT8"

model = LLM(
    model=model_path,    
    gpu_memory_utilization=0.7, # GPU 메모리 70% 사용 (필요시 조절)
    tensor_parallel_size=1,   # GPU 1개 사용
    # enforce_eager=True      # 호환성 모드 켜기 (필요시)
)

sampling_params = SamplingParams(max_tokens=256)

outputs = model.generate("What is machine learning?", sampling_params)

for output in outputs:
    print(output.outputs[0].text)

INFO 01-22 11:08:43 [utils.py:263] non-default args: {'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'model': './TinyLlama-1.1B-Chat-v1.0-INT8'}
INFO 01-22 11:08:43 [model.py:530] Resolved architecture: LlamaForCausalLM
INFO 01-22 11:08:43 [model.py:1545] Using max model len 2048
INFO 01-22 11:08:44 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-22 11:08:44 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-22 11:08:44 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.
(EngineCore_DP0 pid=925) INFO 01-22 11:08:44 [core.py:97] Initializing a V1 LLM engine (v0.14.0) with config: model='./TinyLlama-1.1B-Chat-v1.0-INT8', speculative_config=None, tokenizer='./TinyLlama-1.1B-Chat-v1.0-INT8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pi

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=925) INFO 01-22 11:08:46 [default_loader.py:291] Loading weights took 0.21 seconds
(EngineCore_DP0 pid=925) INFO 01-22 11:08:46 [gpu_model_runner.py:3905] Model loading took 1.15 GiB memory and 0.460047 seconds
(EngineCore_DP0 pid=925) INFO 01-22 11:08:51 [backends.py:644] Using cache directory: /root/.cache/vllm/torch_compile_cache/c0332d605c/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=925) INFO 01-22 11:08:51 [backends.py:704] Dynamo bytecode transform time: 4.47 s
(EngineCore_DP0 pid=925) INFO 01-22 11:08:55 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=925) INFO 01-22 11:08:59 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 6.62 s
(EngineCore_DP0 pid=925) INFO 01-22 11:08:59 [monitor.py:34] torch.compile takes 11.09 s in total
(EngineCore_DP0 pid=925) INFO 01-22 11:09:00 [gpu_worker.py:358] Available KV cache memory: 14.91 GiB
(EngineCore_DP0 pid=925) INFO 01-22 11:09:01 [k

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:00<00:00, 61.46it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:00<00:00, 60.04it/s]


(EngineCore_DP0 pid=925) INFO 01-22 11:09:03 [gpu_model_runner.py:4856] Graph capturing finished in 2 secs, took 0.44 GiB
(EngineCore_DP0 pid=925) INFO 01-22 11:09:03 [core.py:273] init engine (profile, create kv cache, warmup model) took 16.58 seconds
INFO 01-22 11:09:03 [llm.py:347] Supported tasks: ['generate']


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


2.2 How does machine learning work in image classification?
2.3 How does machine learning perform better than human classification algorithms?
2.4 How is model selection and evaluation performed in automated machine learning systems?
2.5 What are the pros and cons of using machine learning algorithms for image classification?
3.1 How is data preprocessing used in machine learning algorithms?
3.2 What is the DL4J framework for data preprocessing in machine learning?
3.3 How does the KNN algorithm perform in data preprocessing and classification tasks?
3.4 How does the Naive Bayes algorithm perform in data preprocessing and classification tasks?
3.5 What sets Noah's Alternative framework apart from other machine learning frameworks for image classification?

Learning outcomes:
1. Understanding the definition of machine learning, its various types of algorithms, and their applications.
2. Understanding how machine learning works in image classification.
3. Identifying the role of preproc

## 2.2. Pruning

In [5]:
# 2:4 sparse pruning with/without FP8 quantization
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.obcq import SparseGPTModifier
from llmcompressor.modifiers.quantization import QuantizationModifier
from llmcompressor.utils import dispatch_for_generation


# Configuration
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
DATASET_ID = "HuggingFaceH4/ultrachat_200k"
DATASET_SPLIT = "train_sft"
NUM_CALIBRATION_SAMPLES = 512
MAX_SEQUENCE_LENGTH = 2048
QUANT_ENABLE = False

def preprocess(example):
    """Preprocess dataset examples."""
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False)}


def tokenize(sample):
    """Tokenize dataset examples."""
    return tokenizer(
        sample["text"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )


def get_recipe(fp8_enabled):
    """
    Generate the compression recipe and save directory based on the FP8 flag.
    """
    
    base_recipe = [
        SparseGPTModifier(
            sparsity=0.5,
            mask_structure="2:4",
            targets=[r"re:model.layers.\d*$"],
        )
    ]
    save_dir = MODEL_ID.rstrip("/").split("/")[-1] + "2of4-sparse"

    if fp8_enabled:
        base_recipe.append(
            QuantizationModifier(
                targets=["Linear"],
                ignore=["lm_head"],
                scheme="FP8_DYNAMIC",
            )
        )
        save_dir = (
            MODEL_ID.rstrip("/").split("/")[-1] + "2of4-W8A8-FP8-Dynamic-Per-Token"
        )

        # check that asymmetric quantization is not being used
        q_scheme = base_recipe[1].scheme
        if not isinstance(q_scheme, str) and not q_scheme["weights"].symmetric:
            raise ValueError(
                "Asymmetric quantization with 2of4 sparsity is not supported by vLLM. "
                "Please use symmetric quantization"
            )

    return base_recipe, save_dir

In [ ]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# Load and preprocess dataset
ds = load_dataset(
    DATASET_ID, 
    split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]"
).shuffle(seed=47)
ds = ds.map(preprocess)
ds = ds.map(tokenize, remove_columns=ds.column_names)

# Get compression recipe and save directory
recipe, save_dir = get_recipe(QUANT_ENABLE)

# Apply compression
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

# Validate the compressed model
print("\n========== SAMPLE GENERATION ==============")
dispatch_for_generation(model)
input_ids = tokenizer("Hello my name is", return_tensors="pt").input_ids.to(
    model.device
)
output = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0]))
print("==========================================\n")

# Save compressed model and tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

In [ ]:
# Inference pruned model via vLLM
from vllm import LLM, SamplingParams


model_path = r"Llama-3.2-3B-Instruct2of4-sparse"

model = LLM(
    model=model_path,    
    gpu_memory_utilization=0.7, # GPU 메모리 70% 사용 (필요시 조절)
    tensor_parallel_size=1,   # GPU 1개 사용
    enforce_eager=True,      # 호환성 모드 켜기 (필요시)
    dtype='auto'
)

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=256
    )

prompt = "Hello, My name is:"

outputs = model.generate(prompt, sampling_params)

for output in outputs:
    print(output.outputs[0].text)